# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kununurra,-15.7667,128.7333,93.18,10,75,10.36,AU,1726287683
1,1,papatowai,-46.5619,169.4708,50.07,64,70,3.69,NZ,1726287684
2,2,iskateley,67.6803,53.1512,62.10,82,52,8.43,RU,1726287685
3,3,labrador city,52.9463,-66.9114,42.19,74,0,11.01,CA,1726287686
4,4,canton,23.1167,113.2500,90.03,61,38,5.21,CN,1726287455


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot

humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    size = "Humidity",
    c = "black",
    alpha = 0.7

)
##### Do the title and maybe legend? different color for different cities? ########

# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values

city_data_df = city_data_df.loc[
    (city_data_df["Max Temp"] <= 80) & 
    (city_data_df["Max Temp"] >= 65) & 
    (city_data_df["Humidity"] <= 40) &
    (city_data_df["Cloudiness"] <= 25) &
    (city_data_df["Wind Speed"] <= 5)
]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df.head(20)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
115,115,aral,40.5156,81.2636,76.08,23,1,3.38,CN,1726287815
119,119,gerdzhyush,37.5651,41.3916,73.49,25,0,3.74,TR,1726287819
121,121,nova xavantina,-14.6667,-52.3500,78.98,30,0,1.50,BR,1726287821
135,135,eregli,37.5133,34.0467,67.80,40,0,2.71,TR,1726287836
152,152,panjakent,39.4952,67.6093,71.51,21,0,1.61,TJ,1726287858
249,249,qazax,41.0933,45.3661,67.82,38,0,4.72,AZ,1726287978
250,250,hotan,37.0997,79.9269,76.66,16,7,3.51,CN,1726287979
345,345,ribas do rio pardo,-20.4431,-53.7592,77.92,28,8,3.44,BR,1726288090
439,439,hwange,-18.3693,26.5019,67.73,39,1,3.15,ZW,1726288193
464,464,robore,-18.3333,-59.7500,78.98,31,17,3.13,BO,1726288220


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = city_data_df.copy(deep=True)


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
115,115,aral,40.5156,81.2636,76.08,23,1,3.38,CN,1726287815,
119,119,gerdzhyush,37.5651,41.3916,73.49,25,0,3.74,TR,1726287819,
121,121,nova xavantina,-14.6667,-52.3500,78.98,30,0,1.50,BR,1726287821,
135,135,eregli,37.5133,34.0467,67.80,40,0,2.71,TR,1726287836,
152,152,panjakent,39.4952,67.6093,71.51,21,0,1.61,TJ,1726287858,
249,249,qazax,41.0933,45.3661,67.82,38,0,4.72,AZ,1726287978,
250,250,hotan,37.0997,79.9269,76.66,16,7,3.51,CN,1726287979,
345,345,ribas do rio pardo,-20.4431,-53.7592,77.92,28,8,3.44,BR,1726288090,
439,439,hwange,-18.3693,26.5019,67.73,39,1,3.15,ZW,1726288193,
464,464,robore,-18.3333,-59.7500,78.98,31,17,3.13,BO,1726288220,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
radius = 10000
limit = 20
categories = "accommodation.hotel"
params = {
    "limit":limit,
    "apiKey":geoapify_key,
    "categories": categories
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = hotel_df.loc[index,"Lng"]
    latitude = hotel_df.loc[index,"Lat"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
        
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
{'type': 'FeatureCollection', 'features': []}
aral - nearest hotel: No hotel found
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Grand Lavin Otel', 'country': 'Turkey', 'country_code': 'tr', 'state': 'Southeastern Anatolia Region', 'county': 'Batman', 'city': 'Gercüş', 'street': 'Batman-Mardin yolu', 'lon': 41.358423956165254, 'lat': 37.50715265, 'formatted': 'Grand Lavin Otel, Batman-Mardin yolu, Gercüş, Turkey', 'address_line1': 'Grand Lavin Otel', 'address_line2': 'Batman-Mardin yolu, Gercüş, Turkey', 'categories': ['accommodation', 'accommodation.hotel'], 'details': ['details'], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database License', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'name': 'Grand Lavin Otel', 'osm_id': 300422961, 'tourism': 'hotel', 'operator': 'Serhat Sevim', 'osm_type': 'w', 'addr:city': 'Batman', 'addr:district': 'Gercü

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
115,115,aral,40.5156,81.2636,76.08,23,1,3.38,CN,1726287815,No hotel found
119,119,gerdzhyush,37.5651,41.3916,73.49,25,0,3.74,TR,1726287819,Grand Lavin Otel
121,121,nova xavantina,-14.6667,-52.3500,78.98,30,0,1.50,BR,1726287821,Hotel Morada do Sol
135,135,eregli,37.5133,34.0467,67.80,40,0,2.71,TR,1726287836,Nupelda Caddesi
152,152,panjakent,39.4952,67.6093,71.51,21,0,1.61,TJ,1726287858,Umariyon
249,249,qazax,41.0933,45.3661,67.82,38,0,4.72,AZ,1726287978,Qazax Olimpiya kompleksi
250,250,hotan,37.0997,79.9269,76.66,16,7,3.51,CN,1726287979,Hua Yi International Hotel (accepts foreigners)
345,345,ribas do rio pardo,-20.4431,-53.7592,77.92,28,8,3.44,BR,1726288090,Hotel São Francisco
439,439,hwange,-18.3693,26.5019,67.73,39,1,3.15,ZW,1726288193,No hotel found
464,464,robore,-18.3333,-59.7500,78.98,31,17,3.13,BO,1726288220,Lajas


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)